In [1]:
# -----------------------------------------------
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
# -----------------------------------------------

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

from pathlib import Path

from utils.model_factory_image import ModelBuilder
from utils.model_factory_title import build_title_model
from utils.model_factory_dual import concat_image_title_model

from utils.dataset_image import image_dataset
from utils.dataset_text import TextDataset
from utils.dataset_dual import DualDataset

from utils.config import (
    dataset_test_path,
    dataset_train_path,
    dataset_validate_path,
    dataset_images_path)


In [2]:
print('Number of GPUs available:', len(tf.config.experimental.list_physical_devices('GPU')))

Number of GPUs available: 1


In [3]:
root_path = Path().absolute().parent

# Model Evaluation

## Text Model

In [10]:
title_dataset_roberta = TextDataset('test', False).text_dataset()
title_dataset_bert = TextDataset('test', True).text_dataset()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


### Load the models

In [6]:
ROBERTA_MODEL_NAME = 'Text_roBERTa_10K_V3'
BERT_MODEL_NAME = 'Text_BERT_10K_V3'

title_model_roberta = build_title_model(2, False);
title_model_roberta.load_weights(Path(root_path, 'models', f'{ROBERTA_MODEL_NAME}.hdf5'));

title_model_bert = build_title_model(2, True);
title_model_bert.load_weights(Path(root_path, 'models', f'{BERT_MODEL_NAME}.hdf5'));

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-

In [7]:
title_model_roberta.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 124645632   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 128, 100)     327600      tf_roberta_model[0][0]       

In [8]:
title_model_bert.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 108310272   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 128, 100)     327600      tf_bert_model[0][0]        

### Evaluate the models

roBERTa:

In [11]:
results = title_model_roberta.evaluate(title_dataset_roberta, return_dict=True)
print(results)
for metric in ['loss', 'accuracy', 'precision', 'recall', 'f1_score']:
    print(f'{metric}:', results[metric])

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


592/592 [==============================] - 364s 611ms/step - loss: 0.4660 - accuracy: 0.7841 - precision: 0.8180 - recall: 0.7127 - f1_score: 0.7822
{'loss': 0.468016117811203, 'accuracy': 0.7829391956329346, 'precision': 0.8160369396209717, 'recall': 0.7110134959220886, 'f1_score': 0.7808821201324463}
loss: 0.468016117811203
accuracy: 0.7829391956329346
precision: 0.8160369396209717
recall: 0.7110134959220886
f1_score: 0.7808821201324463


BERT:

In [12]:
results = title_model_bert.evaluate(title_dataset_bert, return_dict=True)
print()
for metric in ['loss', 'accuracy', 'precision_1', 'recall_1', 'f1_score']:
    print(f'{metric}:', results[metric])

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


592/592 [==============================] - 365s 612ms/step - loss: 0.4335 - accuracy: 0.7987 - precision_1: 0.8048 - recall_1: 0.7840 - f1_score: 0.7911

loss: 0.4365513026714325
accuracy: 0.7967567443847656
precision_1: 0.8032341003417969
recall_1: 0.7836824059486389
f1_score: 0.7890773415565491


## Image model

In [4]:
image_dataset_test = image_dataset('test')

In [5]:
EFFICIENTNET_IMAGENET_NAME = 'EfficientNET_B3_10K_imagenet_V1'
EFFICIENTNET_NOISYSTUDENT_NAME = 'EfficientNET_B3_10K_noisy_student_V2'

EFFICIENTNET_IMAGENET_PATH = Path(root_path, 'models', f'{EFFICIENTNET_IMAGENET_NAME}.hdf5')
EFFICIENTNET_NOISYSTUDENT_PATH = Path(root_path, 'models', f'{EFFICIENTNET_NOISYSTUDENT_NAME}.hdf5')

imagenet = ModelBuilder('b3')
imagenet.compile_for_transfer_learning()
imagenet.model.load_weights(EFFICIENTNET_IMAGENET_PATH)
imagenet = imagenet.model

noisy_student = ModelBuilder('b3')
noisy_student.compile_for_transfer_learning()
noisy_student.model.load_weights(EFFICIENTNET_NOISYSTUDENT_PATH)
noisy_student = noisy_student.model

In [6]:
imagenet.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 380, 380, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 380, 380, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

### ImageNet

In [8]:
results = imagenet.evaluate(image_dataset_test, return_dict=True)
print()
for metric in ['loss', 'accuracy', 'precision', 'recall', 'f1_score']:
    print(f'{metric}:', results[metric])

1481/1481 [==============================] - 530s 355ms/step - loss: 0.5850 - accuracy: 0.7715 - precision: 0.7715 - recall: 0.7715 - f1_score: 0.7696

loss: 0.5868998169898987
accuracy: 0.770965576171875
precision: 0.770965576171875
recall: 0.770965576171875
f1_score: 0.7690004110336304


### EfficientNET

In [9]:
results = noisy_student.evaluate(image_dataset_test, return_dict=True)
print()
for metric in ['loss', 'accuracy', 'precision', 'recall', 'f1_score']:
    print(f'{metric}:', results[metric])

1481/1481 [==============================] - 533s 358ms/step - loss: 0.6686 - accuracy: 0.7739 - precision: 0.7739 - recall: 0.7739 - f1_score: 0.7734

loss: 0.6720579266548157
accuracy: 0.772045910358429
precision: 0.772045910358429
recall: 0.772045910358429
f1_score: 0.7714598178863525


## Dual model

In [4]:
dual_roberta_test = DualDataset('test', False).dual_dataset()
dual_bert_test = DualDataset('test', True).dual_dataset()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


**Efficient B3, ImageNet + roBERTa**

In [6]:
dual_model_name = 'Dual_10K_roBERTa_EfficientNET_B3_imagenet_V2'
weights = Path(root_path, 'models', f'{dual_model_name}.hdf5')

image_model = ModelBuilder('b3')
image_model.compile_for_transfer_learning()
image_model = image_model.model

title_model = build_title_model(2, False)

dual_model = concat_image_title_model(image_model, title_model, 2)
dual_model.load_weights(weights)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [9]:
dual_model.evaluate(dual_roberta_test, return_dict=True)

592/592 [==============================] - 866s 1s/step - loss: 0.4615 - accuracy: 0.8318 - precision_1: 0.8318 - recall_1: 0.8318 - f1_score: 0.8303


{'loss': 0.46149688959121704,
 'accuracy': 0.831790566444397,
 'precision_1': 0.831790566444397,
 'recall_1': 0.831790566444397,
 'f1_score': 0.8302531242370605}

**Efficient B3, ImageNet + BERT**

In [16]:
dual_model_name = 'Dual_10K_BERT_EfficientNET_B3_imagenet_V2'
weights = Path(root_path, 'models', f'{dual_model_name}.hdf5')

image_model = ModelBuilder('b3')
image_model.compile_for_transfer_learning()
image_model = image_model.model

title_model = build_title_model(2, True)

dual_model = concat_image_title_model(image_model, title_model, 2)
dual_model.load_weights(weights)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config objec

In [17]:
dual_model.evaluate(dual_bert_test, return_dict=True)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


592/592 [==============================] - 965s 2s/step - loss: 0.3986 - accuracy: 0.8400 - precision_13: 0.8400 - recall_13: 0.8400 - f1_score: 0.8350


{'loss': 0.39593812823295593,
 'accuracy': 0.8414865136146545,
 'precision_13': 0.8414865136146545,
 'recall_13': 0.8414865136146545,
 'f1_score': 0.8366014957427979}

**Efficient B3, Noisy Student + roBERTa**

In [5]:
dual_model_name = 'Dual_10K_roBERTa_EfficientNET_B3_noisy_student_V2'
weights = Path(root_path, 'models', f'{dual_model_name}.hdf5')

image_model = ModelBuilder('b3')
image_model.compile_for_transfer_learning()
image_model = image_model.model

title_model = build_title_model(2, False)

dual_model = concat_image_title_model(image_model, title_model, 2)
dual_model.load_weights(weights)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [6]:
dual_model.evaluate(dual_roberta_test, return_dict=True)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1185/1185 [==============================] - 1561s 1s/step - loss: 0.4143 - accuracy: 0.8363 - precision_1: 0.8363 - recall_1: 0.8363 - f1_score: 0.8341


{'loss': 0.41108453273773193,
 'accuracy': 0.840286910533905,
 'precision_1': 0.840286910533905,
 'recall_1': 0.840286910533905,
 'f1_score': 0.8380711078643799}

**Efficient B3, Noisy Student + BERT**

In [7]:
dual_model_name = 'Dual_10K_BERT_EfficientNET_B3_noisy_student_V2'
weights = Path(root_path, 'models', f'{dual_model_name}.hdf5')

image_model = ModelBuilder('b3')
image_model.compile_for_transfer_learning()
image_model = image_model.model

title_model = build_title_model(2, True)

dual_model = concat_image_title_model(image_model, title_model, 2)
dual_model.load_weights(weights)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config objec

In [8]:
dual_model.evaluate(dual_roberta_test, return_dict=True)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1185/1185 [==============================] - 2699s 2s/step - loss: 0.4821 - accuracy: 0.7932 - precision_3: 0.7932 - recall_3: 0.7932 - f1_score: 0.7852


{'loss': 0.48140764236450195,
 'accuracy': 0.7918143272399902,
 'precision_3': 0.7918143272399902,
 'recall_3': 0.7918143272399902,
 'f1_score': 0.783739447593689}